##### ! pip install pymysql 설치

In [1]:
import pymysql
import pandas as pd
import smtplib, ssl

#####  DB  생성 (CREATE 사용)

In [12]:
def make_DB(password, DB_name):
    
    conn = pymysql.connect(host='localhost', 
                           user = 'root', 
                           password=password, 
                           charset='utf8',
                           cursorclass=pymysql.cursors.DictCursor)
    try:
        with conn.cursor() as cursor:
            print('creating DataBase ...')
            sql = 'CREATE DATABASE {}'.format(DB_name)
            cursor.execute(sql)
        conn.commit()
    finally:
        conn.close()

#####  table  생성 (CREATE 사용)

In [3]:
def make_table(password, DB_name, table_name, table_colums):
    
    conn = pymysql.connect(host='localhost', 
                           user = 'root', 
                           password=password, 
                           db=DB_name, 
                           charset='utf8',
                           cursorclass=pymysql.cursors.DictCursor)
    try:
        with conn.cursor() as cursor:
            print('creating table ...')
            
            # 테이블이 생성되지 않은 경우만 테이블 생성
            sql = 'CREATE TABLE IF NOT EXISTS {} {}'.format(table_name, table_colums)
            cursor.execute(sql)
        conn.commit()
    finally:
        conn.close()    

#####  table  전체 삭제 (DROP 사용)

In [4]:
# 테이블 내용 전체 삭제 

def drop_table(password, DB_name, table_name):
    conn = pymysql.connect(host='localhost', 
                           user = 'root', 
                           password=password, 
                           db=DB_name, 
                           charset='utf8',
                           cursorclass=pymysql.cursors.DictCursor)
    try:
        with conn.cursor() as cursor:
            print('delete table ...')
            sql  = "DROP TABLE IF EXISTS {}".format(table_name)
            cursor.execute(sql)
            conn.commit()
            
    finally:
        conn.close()  

##### data 입력 ( INSERT 사용)

In [5]:
def input_data(password, DB_name, table_name, Data):
 
    conn = pymysql.connect(host='localhost', 
                           user = 'root', 
                           password=password, 
                           db=DB_name, 
                           charset='utf8',
                           cursorclass=pymysql.cursors.DictCursor)
    try:
        with conn.cursor() as cursor:
            print('inserting data ...')

            # FOR LOOP로 바꿔서 수집한 데이터 테이블에  INSERT하기
            
            #### 매번 다른 사이즈의 데이터에 맞춰 어떻게 바꿀지????????????????????????????
            
            sql = "INSERT INTO {} VALUES (%s, %s, %s, %s, %s)".format(table_name)
            for index, row in Data.iterrows():

                val = (row['날짜'], row['USA'], row['EUR'], row['JPY'], row['CNY'])
#                 val = (row['날짜'], row['title'], row['category'], row['summary'], row['img'])

                try:
                    cursor.execute(sql, val)

                except:
                    print(cursor._last_executed)
    
        conn.commit()
        print(cursor.rowcount, "record inserted.")
    finally:
        conn.close()    

##### data 조회 (SELECT 사용)

In [6]:
def data_select(password,DB_name, table_name):

    conn = pymysql.connect(host='localhost', 
                               user = 'root', 
                               password=password, 
                               db=DB_name, 
                               charset='utf8',
                               cursorclass=pymysql.cursors.DictCursor)
    try:
        with conn.cursor() as cursor:
            sql = "SELECT * FROM {}".format(table_name)
            cursor.execute(sql)
            
            query_result = pd.read_sql (sql, conn)
#             result = cursor.fetchall()
#             df = pd.DataFrame(result)
            return (df)

    finally:
        conn.close()

##### 구글 이메일 보내기

In [7]:
def send_email(msg):
    port = 465  # For SSL
    smtp_server = "smtp.gmail.com"
    sender_email = "leejm456@gmail.com"  # Enter your address
    receiver_email = "leejm456@gmail.com"  # Enter receiver address
    password = 'mpzzchhaznjfwpuc' #input("Type your password and press enter: ")
    message = str(msg)
    context = ssl.create_default_context()
    with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message.encode('utf-8') )

#####  main.py

In [13]:
input_date = input('정보를 받고 싶은 날짜를 입력하세요 eg) 2020-06-08  ')

file_list = ['./naver_exchange_crawling.csv', './finally_naver_news_crawling.csv']
table_name_list = ['naver_exchange','naver_ranking_news']
table_columns_list =["(날짜 CHAR(10) PRIMARY KEY,USA float(10) NOT NULL, EUR float(10) NOT NULL,JPY float(10) NOT NULL,CNY float(10) NOT NULL)","(날짜 CHAR(10), Title VARCHAR(30) NOT NULL, category VARCHAR(5) NOT NULL,summary VARCHAR(255),img VARCHAR(255))"] 



for file_path, name, columns in zip(file_list, table_name_list, table_columns_list):
    print('-'*100)
    Data = pd.read_csv(file_path)

    password = '1234'
    DB_name = 'multi'
    table_name = name
    table_columns = columns


    # DB 생성
#     make_DB(password, DB_name)

    # table 생성
    make_table(password, DB_name, table_name, table_columns)

    # data 입력
    input_data(password, DB_name, table_name, Data)

    # data 조회
    output_data = data_select(password, DB_name, table_name)

    print(output_data)
    
# exchange_msg = output_data[output_data['날짜']== input_date] 
# print(body)

# 이메일 보내기
# send_email(msg)


정보를 받고 싶은 날짜를 입력하세요 eg) 2020-06-08  2020-06-08
----------------------------------------------------------------------------------------------------


InternalError: (1049, "Unknown database 'multi'")